In [8]:
import os
import sys
import pytest
import numpy as np
import pandas as pd
path = os.path.realpath('temp.ipynb')
sys.path.append(os.path.dirname(os.path.dirname(path)))
import deployment_scripts as dep

In [9]:
ad_reactors = {
    'ReactorBig': [80, 1, 6, [1,2,1,2,1,2,1,2,1]],
    'ReactorMedium':[20,1,4,'no_dist'],
    'ReactorSmall': [5, 1, 2, 'no_dist']}
# {reactor: [Power (MWe), capacity_factor (%), lifetime (yr), distribution
# (default='no_dist')]}

test_dict = {
    'Year':[2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024],
    'test_cap': [20, 79, 80, 81, 220, 640, 693, 950, 700]}
test_df = pd.DataFrame.from_dict(test_dict)

In [10]:
greedy_dep_df = test_df.copy()
dep.greedy_deployment(greedy_dep_df, 'test_cap', ad_reactors)

,Year,test_cap,num_ReactorBig,num_ReactorMedium,num_ReactorSmall,ReactorBigDecom,ReactorMediumDecom,ReactorSmallDecom,total_cap,new_cap,new_ReactorBig_cap,ReactorBig_cap,new_ReactorMedium_cap,ReactorMedium_cap,new_ReactorSmall_cap,ReactorSmall_cap
0,2016,20,0,1,0,0,0,0,20,20,0,0,20,20,0,0
1,2017,79,0,3,3,0,0,0,75,75,0,0,60,60,15,15
2,2018,80,1,0,0,0,0,0,80,80,80,80,0,0,0,0
3,2019,81,1,0,3,0,0,3,95,80,80,80,0,0,0,15
4,2020,220,2,4,0,0,1,0,240,220,160,160,60,80,0,0
5,2021,640,8,3,3,0,3,3,715,640,640,640,0,60,0,15
6,2022,693,8,2,2,0,0,0,690,690,640,640,40,40,10,10
7,2023,950,11,3,5,0,0,3,965,950,880,880,60,60,10,25
8,2024,700,9,7,2,1,4,2,870,700,640,720,60,140,0,10


In [11]:
greedy_dep_df[['Year','test_cap','new_cap']].to_dict()

{'Year': {0: 2016,
  1: 2017,
  2: 2018,
  3: 2019,
  4: 2020,
  5: 2021,
  6: 2022,
  7: 2023,
  8: 2024},
 'test_cap': {0: 20,
  1: 79,
  2: 80,
  3: 81,
  4: 220,
  5: 640,
  6: 693,
  7: 950,
  8: 700},
 'new_cap': {0: 20,
  1: 75,
  2: 80,
  3: 80,
  4: 220,
  5: 640,
  6: 690,
  7: 950,
  8: 700}}

In [12]:
react_to_cap_df = {
        'Year': [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024],
        'manual_cap': [0, 0, 80, 80, 160, 640, 640, 880, 720],
        'new_cap': [0, 0, 80, 80, 160, 640, 640, 880, 720]}
print(react_to_cap_df['manual_cap'][i] == react_to_cap_df['new_cap'][i] for i in range(len(react_to_cap_df['manual_cap'])))



<generator object <genexpr> at 0x7fe7081b4f20>


In [13]:
# Test direct_decom and num_react_to_cap
test_cap_reactors = {'ReactorBig': [80, 1, 6, [1,2,1,2,1,2,1,2,1]]}
test_cap_df = greedy_dep_df[['Year','num_ReactorBig']].copy()
test_cap_df['manual_cap'] = test_cap_df['num_ReactorBig'] * ad_reactors['ReactorBig'][0]
test_cap_df['manual_decom'] = 0
test_cap_df.loc[2 + test_cap_reactors['ReactorBig'][2],'manual_decom'] = 1
test_cap_df = dep.direct_decom(test_cap_df, test_cap_reactors)
test_cap_df = dep.num_react_to_cap(test_cap_df, test_cap_reactors)
test_cap_df

,Year,num_ReactorBig,manual_cap,manual_decom,ReactorBigDecom,total_cap,new_cap,new_ReactorBig_cap,ReactorBig_cap
0,2016,0,0,0,0,0,0,0,0
1,2017,0,0,0,0,0,0,0,0
2,2018,1,80,0,0,80,80,80,80
3,2019,1,80,0,0,80,80,80,80
4,2020,2,160,0,0,160,160,160,160
5,2021,8,640,0,0,640,640,640,640
6,2022,8,640,0,0,640,640,640,640
7,2023,11,880,0,0,880,880,880,880
8,2024,10,720,1,1,800,720,720,800


In [14]:
test_cap_df[['Year','manual_decom','ReactorBigDecom']].to_dict()

{'Year': {0: 2016,
  1: 2017,
  2: 2018,
  3: 2019,
  4: 2020,
  5: 2021,
  6: 2022,
  7: 2023,
  8: 2024},
 'manual_decom': {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1},
 'ReactorBigDecom': {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1}}

In [15]:
test_cap_df['manual_']

KeyError: 'manual_'

In [ ]:
pre_det_dep_df_greedy = test_df.copy()
dep.pre_det_deployment(pre_det_dep_df_greedy, 'test_cap', ad_reactors)

,Year,test_cap,num_ReactorBig,num_ReactorMedium,num_ReactorSmall,ReactorBigDecom,ReactorMediumDecom,ReactorSmallDecom,total_cap,new_cap,new_ReactorBig_cap,ReactorBig_cap,new_ReactorMedium_cap,ReactorMedium_cap,new_ReactorSmall_cap,ReactorSmall_cap
0,2016,20,0,1,0,0,0,0,20,20,0,0,20,20,0,0
1,2017,79,0,3,3,0,0,0,75,75,0,0,60,60,15,15
2,2018,80,1,0,0,0,0,0,80,80,80,80,0,0,0,0
3,2019,81,1,0,3,0,0,3,95,80,80,80,0,0,0,15
4,2020,220,1,8,0,0,1,0,240,220,80,80,140,160,0,0
5,2021,640,2,27,3,0,3,3,715,640,160,160,480,540,0,15
6,2022,693,1,30,2,0,0,0,690,690,80,80,600,600,10,10
7,2023,950,2,39,5,0,0,3,965,950,160,160,780,780,10,25
8,2024,700,2,39,2,1,8,2,950,700,80,160,620,780,0,10


In [ ]:
pre_det_dep_df_not = test_df.copy()
dep.pre_det_deployment(pre_det_dep_df_not, 'test_cap', ad_reactors, False)

,Year,test_cap,num_ReactorBig,num_ReactorMedium,num_ReactorSmall,ReactorBigDecom,ReactorMediumDecom,ReactorSmallDecom,total_cap,new_cap,new_ReactorBig_cap,ReactorBig_cap,new_ReactorMedium_cap,ReactorMedium_cap,new_ReactorSmall_cap,ReactorSmall_cap
0,2016,20,1,0,0,0,0,0,80,80,80,80,0,0,0,0
1,2017,79,1,0,0,0,0,0,80,80,80,80,0,0,0,0
2,2018,80,1,1,0,0,0,0,100,100,80,80,20,20,0,0
3,2019,81,1,1,0,0,0,0,100,100,80,80,20,20,0,0
4,2020,220,1,6,5,0,0,0,225,225,80,80,120,120,25,25
5,2021,640,2,20,19,0,0,0,655,655,160,160,400,400,95,95
6,2022,693,2,26,29,1,1,5,825,700,80,160,500,520,120,145
7,2023,950,3,33,50,1,1,19,1150,955,160,240,640,660,155,250
8,2024,700,2,31,54,1,6,29,1050,705,80,160,500,620,125,270


In [16]:
pre_det_dep_df_greedy = test_df.copy()
dep.pre_det_deployment(pre_det_dep_df_greedy, 'test_cap', ad_reactors)

,Year,test_cap,num_ReactorBig,num_ReactorMedium,num_ReactorSmall,ReactorBigDecom,ReactorMediumDecom,ReactorSmallDecom,total_cap,new_cap,new_ReactorBig_cap,ReactorBig_cap,new_ReactorMedium_cap,ReactorMedium_cap,new_ReactorSmall_cap,ReactorSmall_cap
0,2016,20,0,1,0,0,0,0,20,20,0,0,20,20,0,0
1,2017,79,0,3,3,0,0,0,75,75,0,0,60,60,15,15
2,2018,80,1,0,0,0,0,0,80,80,80,80,0,0,0,0
3,2019,81,1,0,3,0,0,3,95,80,80,80,0,0,0,15
4,2020,220,1,8,0,0,1,0,240,220,80,80,140,160,0,0
5,2021,640,2,27,3,0,3,3,715,640,160,160,480,540,0,15
6,2022,693,1,30,2,0,0,0,690,690,80,80,600,600,10,10
7,2023,950,2,39,5,0,0,3,965,950,160,160,780,780,10,25
8,2024,700,2,39,2,1,8,2,950,700,80,160,620,780,0,10


In [18]:
pre_det_dep_df_greedy[['Year','test_cap','total_cap','new_cap']].to_dict()

{'Year': {0: 2016,
  1: 2017,
  2: 2018,
  3: 2019,
  4: 2020,
  5: 2021,
  6: 2022,
  7: 2023,
  8: 2024},
 'test_cap': {0: 20,
  1: 79,
  2: 80,
  3: 81,
  4: 220,
  5: 640,
  6: 693,
  7: 950,
  8: 700},
 'total_cap': {0: 20,
  1: 75,
  2: 80,
  3: 95,
  4: 240,
  5: 715,
  6: 690,
  7: 965,
  8: 950},
 'new_cap': {0: 20,
  1: 75,
  2: 80,
  3: 80,
  4: 220,
  5: 640,
  6: 690,
  7: 950,
  8: 700}}

In [14]:
pre_det_dep_df_linear = test_df.copy()
dep.pre_det_deployment(pre_det_dep_df_linear, 'test_cap', ad_reactors, False)

,Year,test_cap,num_ReactorBig,num_ReactorMedium,num_ReactorSmall,ReactorBigDecom,ReactorMediumDecom,ReactorSmallDecom,total_cap,new_cap,new_ReactorBig_cap,ReactorBig_cap,new_ReactorMedium_cap,ReactorMedium_cap,new_ReactorSmall_cap,ReactorSmall_cap
0,2016,20,1,0,0,0,0,0,80,80,80,80,0,0,0,0
1,2017,79,1,0,0,0,0,0,80,80,80,80,0,0,0,0
2,2018,80,1,1,0,0,0,0,100,100,80,80,20,20,0,0
3,2019,81,1,1,0,0,0,0,100,100,80,80,20,20,0,0
4,2020,220,1,6,5,0,0,0,225,225,80,80,120,120,25,25
5,2021,640,2,20,19,0,0,0,655,655,160,160,400,400,95,95
6,2022,693,2,26,29,1,1,5,825,700,80,160,500,520,120,145
7,2023,950,3,33,50,1,1,19,1150,955,160,240,640,660,155,250
8,2024,700,2,31,54,1,6,29,1050,705,80,160,500,620,125,270


In [15]:
pre_det_dep_df_linear[['test_cap','total_cap','new_cap']].to_dict()

{'test_cap': {0: 20,
  1: 79,
  2: 80,
  3: 81,
  4: 220,
  5: 640,
  6: 693,
  7: 950,
  8: 700},
 'total_cap': {0: 80,
  1: 80,
  2: 100,
  3: 100,
  4: 225,
  5: 655,
  6: 825,
  7: 1150,
  8: 1050},
 'new_cap': {0: 80,
  1: 80,
  2: 100,
  3: 100,
  4: 225,
  5: 655,
  6: 700,
  7: 955,
  8: 705}}

In [17]:
## test deployment cap extension
cap_reactors = ad_reactors.copy()
cap_reactors['ReactorBig'][3] = 9*[2]

pre_cap_dep_df_linear = test_df.copy()
dep.pre_det_deployment(pre_cap_dep_df_linear, 'test_cap', cap_reactors, False)

,Year,test_cap,num_ReactorBig,num_ReactorMedium,num_ReactorSmall,ReactorBigDecom,ReactorMediumDecom,ReactorSmallDecom,total_cap,new_cap,new_ReactorBig_cap,ReactorBig_cap,new_ReactorMedium_cap,ReactorMedium_cap,new_ReactorSmall_cap,ReactorSmall_cap
0,2016,20,1,0,0,0,0,0,80,80,80,80,0,0,0,0
1,2017,79,1,0,0,0,0,0,80,80,80,80,0,0,0,0
2,2018,80,1,1,0,0,0,0,100,100,80,80,20,20,0,0
3,2019,81,1,1,0,0,0,0,100,100,80,80,20,20,0,0
4,2020,220,2,3,2,0,0,0,230,230,160,160,60,60,10,10
5,2021,640,2,20,19,0,0,0,655,655,160,160,400,400,95,95
6,2022,693,3,23,23,1,1,2,815,705,160,240,440,460,105,115
7,2023,950,3,33,50,1,1,19,1150,955,160,240,640,660,155,250
8,2024,700,3,25,44,1,3,23,960,705,160,240,440,500,105,220


In [18]:
cap_reactors

{'ReactorBig': [80, 1, 6, [2, 2, 2, 2, 2, 2, 2, 2, 2]],
 'ReactorMedium': [20, 1, 4, 'no_dist'],
 'ReactorSmall': [5, 1, 2, 'no_dist']}

In [19]:
pre_cap_dep_df_linear[['test_cap','total_cap','new_cap']].to_dict()

{'test_cap': {0: 20,
  1: 79,
  2: 80,
  3: 81,
  4: 220,
  5: 640,
  6: 693,
  7: 950,
  8: 700},
 'total_cap': {0: 80,
  1: 80,
  2: 100,
  3: 100,
  4: 230,
  5: 655,
  6: 815,
  7: 1150,
  8: 960},
 'new_cap': {0: 80,
  1: 80,
  2: 100,
  3: 100,
  4: 230,
  5: 655,
  6: 705,
  7: 955,
  8: 705}}

In [13]:
pre_cap_dep_df_greedy = test_df.copy()
dep.pre_det_deployment(pre_cap_dep_df_greedy, 'test_cap', cap_reactors)

,Year,test_cap,num_ReactorBig,num_ReactorMedium,num_ReactorSmall,ReactorBigDecom,ReactorMediumDecom,ReactorSmallDecom,total_cap,new_cap,new_ReactorBig_cap,ReactorBig_cap,new_ReactorMedium_cap,ReactorMedium_cap,new_ReactorSmall_cap,ReactorSmall_cap
0,2016,20,0,1,0,0,0,0,20,20,0,0,20,20,0,0
1,2017,79,0,3,3,0,0,0,75,75,0,0,60,60,15,15
2,2018,80,1,0,0,0,0,0,80,80,80,80,0,0,0,0
3,2019,81,1,0,3,0,0,3,95,80,80,80,0,0,0,15
4,2020,220,2,4,0,0,1,0,240,220,160,160,60,80,0,0
5,2021,640,2,27,3,0,3,3,715,640,160,160,480,540,0,15
6,2022,693,2,26,2,0,0,0,690,690,160,160,520,520,10,10
7,2023,950,2,39,5,0,0,3,965,950,160,160,780,780,10,25
8,2024,700,3,31,2,1,4,2,870,700,160,240,540,620,0,10


In [14]:
pre_cap_dep_df_greedy[['test_cap','total_cap','new_cap']]

,test_cap,total_cap,new_cap
0,20,20,20
1,79,75,75
2,80,80,80
3,81,95,80
4,220,240,220
5,640,715,640
6,693,690,690
7,950,965,950
8,700,870,700


In [15]:
# random rough
rand_rough_dep_df = test_df.copy()
dep.rand_deployment(rand_rough_dep_df, 'test_cap', ad_reactors, True)

,Year,test_cap,num_ReactorBig,num_ReactorMedium,num_ReactorSmall,ReactorBigDecom,ReactorMediumDecom,ReactorSmallDecom,total_cap,new_cap,new_ReactorBig_cap,ReactorBig_cap,new_ReactorMedium_cap,ReactorMedium_cap,new_ReactorSmall_cap,ReactorSmall_cap
0,2016,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2017,79,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2018,80,1,0,0,0,0,0,80,80,80,80,0,0,0,0
3,2019,81,1,0,0,0,0,0,80,80,80,80,0,0,0,0
4,2020,220,2,2,1,0,0,0,205,205,160,160,40,40,5,5
5,2021,640,6,6,3,0,0,0,615,615,480,480,120,120,15,15
6,2022,693,6,6,4,0,0,1,620,615,480,480,120,120,15,20
7,2023,950,9,8,7,0,0,3,915,900,720,720,160,160,20,35
8,2024,700,8,8,7,1,2,4,835,695,560,640,120,160,15,35


In [16]:
rand_rough_dep_df[['test_cap','total_cap','new_cap']]

,test_cap,total_cap,new_cap
0,20,0,0
1,79,0,0
2,80,80,80
3,81,80,80
4,220,205,205
5,640,615,615
6,693,620,615
7,950,915,900
8,700,835,695


In [63]:
def rand_greedy_deployment(df, base_col, ar_dict, set_seed):
    """
    This function combines the rough random and greedy deployments to fill in any gaps caused by the roughness.

    Parameters
    ----------
    df: pandas dataframe
        The dataframe of capacity information.
    base_col: str
        The string name corresponding to the column of capacity that the
        algorithm is deploying reactors to meet.
    ar_dict: dictionary
        A dictionary of reactors with information of the form:
        {reactor: [Power (MWe), capacity_factor (%), lifetime (yr)]}.
    set_seed: bool
        A True/False value that determines whether the seed used for the random
        number is set or varies based on time.
    """
    # Initialize the number of reactor columns.
    for reactor in ar_dict.keys():
        if f'num_{reactor}' not in df:
            df[f'num_{reactor}'] = 0
        else:
            pass

    # First we will apply the rough random.
    df = dep.rand_deployment(df, base_col, ar_dict, set_seed, rough=True)

    # Now we will make a remaining cap column.
    df['remaining_cap'] = df[base_col] - df['new_cap']

    # make columns for the randomly deployed reactors and the greedy reactors
    for reactor in ar_dict.keys():
        df[f'greedy_num_{reactor}'] = 0
        df[f'rand_num_{reactor}'] = df[f'num_{reactor}']

    df['new_cap'] = 0

    # Now we will use the remaining cap column with a greedy deployment.
    df = dep.greedy_deployment(df, 'remaining_cap', ar_dict)

    #reset the total capacity column
    df['new_cap'] = 0

    # populate the greedy reactor column
    for year in range(len(df[base_col])):
        for reactor in ar_dict.keys():
            df.loc[year,f'greedy_num_{reactor}'] = df.loc[year,f'num_{reactor}'] - df.loc[year,f'rand_num_{reactor}']
            df.loc[year, f'new_cap'] += df.loc[year,f'{reactor}_cap']

    return df


In [64]:
# random rough with greedy
rand_rough_greedy_dep_df = test_df.copy()
rand_greedy_deployment(rand_rough_greedy_dep_df, 'test_cap', ad_reactors, True)

,Year,test_cap,num_ReactorBig,num_ReactorMedium,num_ReactorSmall,ReactorBigDecom,ReactorMediumDecom,ReactorSmallDecom,total_cap,new_cap,...,ReactorMedium_cap,new_ReactorSmall_cap,ReactorSmall_cap,remaining_cap,greedy_num_ReactorBig,rand_num_ReactorBig,greedy_num_ReactorMedium,rand_num_ReactorMedium,greedy_num_ReactorSmall,rand_num_ReactorSmall
0,2016,20,0,1,0,0,0,0,20,20,...,20,0,0,20,0,0,1,0,0,0
1,2017,79,0,3,3,0,0,0,75,75,...,60,15,15,79,0,0,3,0,3,0
2,2018,80,1,0,0,0,0,0,160,80,...,0,0,0,0,0,1,0,0,0,0
3,2019,81,1,0,3,0,0,3,175,95,...,0,0,15,1,0,1,0,0,3,0
4,2020,220,2,3,4,0,1,0,445,240,...,60,20,20,15,0,2,1,2,3,1
5,2021,640,6,10,7,0,3,3,1330,715,...,200,20,35,25,0,6,4,6,4,3
6,2022,693,6,9,11,0,0,4,1335,715,...,180,35,55,78,0,6,3,6,7,4
7,2023,950,9,10,16,0,0,7,1915,1000,...,200,45,80,50,0,9,2,8,9,7
8,2024,700,9,11,19,1,3,11,1870,1035,...,220,40,95,5,1,8,3,8,12,7


In [65]:
9 * 80 + 11 * 20 + 19*5

1035

In [66]:
rand_rough_greedy_dep_df[['test_cap','total_cap','new_cap']]

,test_cap,total_cap,new_cap
0,20,20,20
1,79,75,75
2,80,160,80
3,81,175,95
4,220,445,240
5,640,1330,715
6,693,1335,715
7,950,1915,1000
8,700,1870,1035


In [61]:
te = {
    # 'Year':[2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024],
    # 'test_cap': [20, 79, 80, 81, 220, 640, 693, 950, 700]
    'Year':[2022, 2023, 2024],
    'test_cap': [693, 950, 700]}
t = pd.DataFrame.from_dict(te)
rand_greedy_deployment(t, 'test_cap', ad_reactors, True)

,Year,test_cap,num_ReactorBig,num_ReactorMedium,num_ReactorSmall,ReactorBigDecom,ReactorMediumDecom,ReactorSmallDecom,total_cap,new_cap,...,ReactorMedium_cap,new_ReactorSmall_cap,ReactorSmall_cap,remaining_cap,greedy_num_ReactorBig,rand_num_ReactorBig,greedy_num_ReactorMedium,rand_num_ReactorMedium,greedy_num_ReactorSmall,rand_num_ReactorSmall
0,2022,693,6,9,6,0,0,0,690,690,...,180,30,30,78,0,6,3,6,3,3
1,2023,950,9,10,6,0,0,0,950,950,...,200,30,30,50,0,9,2,8,2,4
2,2024,700,7,6,13,0,0,6,745,745,...,120,35,65,5,0,7,0,6,7,6


In [24]:
decom_df = {
    'Year': [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024],
    'manual_decom': [0, 0, 0, 0, 0, 0, 0, 0, 1],
    # manual calculation based on greedy algorithm
    'ReactorBigDecom': [0, 0, 0, 0, 0, 0, 0, 0, 1]}
    # result of greedy function using the direct_decom function


def test_direct_decom(decom_df, ad_reactors):

    assert all(decom_df['manual_decom'][i] == decom_df['ReactorBigDecom'][i] for i in range(len(decom_df['manual_decom'])))

In [25]:
test_direct_decom(decom_df, ad_reactors)

True


AssertionError: 